In [47]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to /Users/Sean
[nltk_data]     1/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [48]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [49]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [50]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [51]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [52]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,regular,expenditure,confirm,voice,company,mahogany,queer,especially,discredit,example,...,certain,play,genteel,account,delicate,gift,sensible,break,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [53]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9849624060150376

Test set score: 0.848314606741573


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [54]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(266, 1612) (266,)
Training set score: 0.9774436090225563

Test set score: 0.8707865168539326


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [55]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9774436090225563

Test set score: 0.8146067415730337


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [56]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [57]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [58]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [59]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
Processing row 150
done


In [60]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.7073170731707317


col_0,Austen,Carroll
row_0,,
Austen,155,15
Carroll,57,19


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  


In [61]:
# Group into sentences
# also add number of tokens in each sentence
alice_sents_mod = [[sent, len(sent), "Carroll"] for sent in alice_doc.sents]
persuasion_sents_mod = [[sent, len(sent), "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences_mod = pd.DataFrame(alice_sents_mod + persuasion_sents_mod)
sentences_mod.head()

,0,1,2
0,"(Alice, was, beginning, to, get, very, tired, ...",67,Carroll
1,"(So, she, was, considering, in, her, own, mind...",63,Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",30,Carroll
3,"(Oh, dear, !)",3,Carroll
4,"(Oh, dear, !)",3,Carroll


In [62]:
# Add more token level features
# Let's modify the bow function to include part of speech and dependency
def bag_of_words_mod(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_ + ' ' + token.pos_ + ' ' + token.dep_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    
    
alicewords_mod = bag_of_words_mod(alice_doc)
persuasionwords_mod = bag_of_words_mod(persuasion_doc)
#print(alicewords_mod)

commonwords_mod = set(alicewords_mod + persuasionwords_mod)
len(commonwords_mod)

2507

In [122]:
# We also need to modify the bow_features function to count the words and their added properties
def bow_mod_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df_mod = pd.DataFrame(columns=common_words)
    df_mod['text_sentence'] = sentences[0]
    df_mod['sentence_len'] = sentences[1]
    df_mod['text_source'] = sentences[2]
    df_mod.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df_mod['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_ + ' ' + token.pos_ + ' ' + token.dep_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ + ' ' + token.pos_ + ' ' + token.dep_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df_mod.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df_mod

words_mod_count = bow_mod_features(sentences_mod, commonwords_mod)
words_mod_count.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250


,uxorious ADJ amod,uncompounded ADJ conj,weak ADJ acomp,home NOUN pobj,enormous ADJ amod,worth NOUN pobj,sleep VERB advcl,end NOUN dobj,real ADJ appos,astonishment NOUN nsubj,...,drink VERB advcl,host NOUN pobj,good ADJ advmod,heap NOUN pobj,soft ADJ csubj,pit NOUN pcomp,run VERB advcl,text_sentence,sentence_len,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",67,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,"(So, she, was, considering, in, her, own, mind...",63,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",30,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll


In [123]:
# sentence length column might be problematic so let's put them into categories
# first i need a little more detailed info to see how to categorize
words_mod_count['sentence_len'].describe()

count    286.000000
mean      28.279720
std       23.480012
min        1.000000
25%       10.000000
50%       22.500000
75%       40.000000
max      134.000000
Name: sentence_len, dtype: float64

In [124]:
# median length is 22. So let's say under 18 is short, 19-35 is medium and over 35 is long.
# 0=short, 1=medium, 2=long
condition = [(words_mod_count['sentence_len']<18), 
             (words_mod_count['sentence_len']>18) & (words_mod_count['sentence_len']<36),
             (words_mod_count['sentence_len']>36)]

choice = [0, 1, 2]

words_mod_count['sentence_len'] = np.select(condition, choice)
words_mod_count.head()

,uxorious ADJ amod,uncompounded ADJ conj,weak ADJ acomp,home NOUN pobj,enormous ADJ amod,worth NOUN pobj,sleep VERB advcl,end NOUN dobj,real ADJ appos,astonishment NOUN nsubj,...,drink VERB advcl,host NOUN pobj,good ADJ advmod,heap NOUN pobj,soft ADJ csubj,pit NOUN pcomp,run VERB advcl,text_sentence,sentence_len,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",2,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,"(So, she, was, considering, in, her, own, mind...",2,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",1,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",0,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",0,Carroll


In [129]:
# modify function to include sentence length category
def bow_mod_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df_mod = pd.DataFrame(columns=common_words)
    df_mod['text_sentence'] = sentences[0]
    
    #df_mod['sentence_len'] = sentences[1]
    condition = [(sentences[1]<18), 
                 (sentences[1]>18) & (sentences[1]<36),
                 (sentences[1]>36)]

    choice = [0, 1, 2]

    df_mod['sentence_len'] = np.select(condition, choice)
    
    df_mod['text_source'] = sentences[2]
    
    df_mod.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df_mod['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_ + ' ' + token.pos_ + ' ' + token.dep_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ + ' ' + token.pos_ + ' ' + token.dep_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df_mod.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df_mod

In [118]:
# testing to make sure that the counter is actually working
words_mod_count['company NOUN pobj'].value_counts()

0    442
1      2
Name: company NOUN pobj, dtype: int64

In [130]:
# create training and test sets
X = words_mod_count.drop(['text_sentence', 'text_source'], axis=1)
y = words_mod_count['text_source']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

print(X_train.shape)
print(y_train.shape)

(171, 2332)
(171,)


In [131]:
# let's fit a logistic model
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('Training set: %0.4f' % lr.score(X_train, y_train))
print('Test set: %0.4f' % lr.score(X_test, y_test))


Training set: 0.9708
Test set: 0.8348


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [127]:
# let's try svm
from sklearn.svm import SVC

svc = SVC(gamma='auto')
svc.fit(X_train, y_train)
print('Training set: %0.4f' % svc.score(X_train, y_train))
print('Test set: %0.4f' % svc.score(X_test, y_test))

Training set: 0.5146
Test set: 0.6000


In [128]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set:', clf.score(X_train, y_train))
print('Test set:', clf.score(X_test, y_test))

Training set: 0.9824561403508771
Test set: 0.8260869565217391



# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [68]:
# let's pick out a new book
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [69]:
# I think I'll go with paradise
paradise = gutenberg.raw('milton-paradise.txt')
print('Paradise: {}'.format(len(paradise)))
print('Emma: {}'.format(len(emma)))
print('Alice: {}'.format(len(alice)))
print('Persuasion: {}'.format(len(persuasion)))

Paradise: 468220
Emma: 14617
Alice: 14139
Persuasion: 46284


In [70]:
# portion and clean txt
paradise = re.sub("[\[].*?[\]]", "", paradise)
paradise = re.sub(r"Book [IVX]+", "", paradise)
paradise = text_cleaner(paradise[:int(len(paradise)/20)])
print(paradise[0:100])

Of Man's first disobedience, and the fruit Of that forbidden tree whose mortal taste Brought death i


In [71]:
# parse raw text
paradise_doc = nlp(paradise)

In [77]:
# group into sentences
paradise_sents = [[sent, len(sent), 'Milton'] for sent in paradise_doc.sents]


### Alice vs Paradise

In [84]:
# Combine the sentences from the two novels into one data frame.
alice_paradise_sents = pd.DataFrame(alice_sents_mod + paradise_sents)
alice_paradise_sents.head()

,0,1,2
0,"(Alice, was, beginning, to, get, very, tired, ...",67,Carroll
1,"(So, she, was, considering, in, her, own, mind...",63,Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",30,Carroll
3,"(Oh, dear, !)",3,Carroll
4,"(Oh, dear, !)",3,Carroll


In [85]:
# bow for paradise and alice
paradisewords = bag_of_words_mod(paradise_doc)
alice_paradise_words = set(alicewords_mod + paradisewords)
print(len(alice_paradise_words))

2331


In [133]:
# the number of columns for the df fitted on the model is 2508, while the no of col for in the new df is 
# 2331. We need to reconcile the difference.

# we need to build a model using just the 2300 features to keep it as numerous of possible 
X_2300 = words_mod_count.drop(['text_sentence', 'text_source'], axis=1).sample(2300, axis=1)
y_2300 = words_mod_count['text_source']


X_23_train, X_23_test, y_23_train, y_23_test = train_test_split(X_2300, y_2300, test_size=0.4, random_state=0)

In [134]:
# Now fit model again with slightly smaller df
lr = LogisticRegression()
model = lr.fit(X_23_train, y_23_train)
print('Training set: %0.4f' % model.score(X_23_train, y_23_train))
print('Test set: %0.4f' % model.score(X_23_test, y_23_test))


Training set: 0.9708
Test set: 0.8348


/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [87]:
# get dataframe
alice_paradise_count = bow_mod_features(alice_paradise_sents, alice_paradise_words)
alice_paradise_count.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250


,uxorious ADJ amod,uncompounded ADJ conj,weak ADJ acomp,home NOUN pobj,enormous ADJ amod,worth NOUN pobj,sleep VERB advcl,end NOUN dobj,real ADJ appos,astonishment NOUN nsubj,...,drink VERB advcl,host NOUN pobj,good ADJ advmod,heap NOUN pobj,soft ADJ csubj,pit NOUN pcomp,run VERB advcl,text_sentence,sentence_len,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",67,Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,"(So, she, was, considering, in, her, own, mind...",63,Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",30,Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",3,Carroll


In [135]:
# shape alice+paradise df accordingly
X_alice_paradise = alice_paradise_count.drop(['text_sentence', 'text_source'], axis=1).sample(2300, axis=1)
y_alice_paradise = alice_paradise_count['text_source']

print(X_alice_paradise.shape)
print(y_alice_paradise.shape)

(286, 2300)
(286,)


In [137]:
# test on model
print('Alice v Paradise: %0.4f' % model.score(X_alice_paradise, y_alice_paradise))


Alice v Paradise: 0.4580


In [138]:
y_pred = model.predict(X_alice_paradise)
pd.crosstab(y_alice_paradise, y_pred)

col_0,Carroll,Milton
text_source,,
Carroll,32,97
Milton,58,99


That's pretty terrible.

### Persuasion vs Paradise

In [142]:
# Combine the sentences from the two novels into one data frame.
persu_para_sents = pd.DataFrame(persuasion_sents_mod + paradise_sents)
persu_para_sents.head()

,0,1,2
0,"(Sir, Walter, Elliot, ,, of, Kellynch, Hall, ,...",117,Austen
1,"(This, was, the, page, at, which, the, favouri...",18,Austen
2,"("")",1,Austen
3,"(Walter, Elliot, ,, born, March, 1, ,, 1760, ,...",54,Austen
4,"(Anne, ,, born, August, 9, ,, 1787, ;, a, stil...",27,Austen


In [141]:
# bow for paradise and alice
persu_para_words = set(persuasionwords_mod + paradisewords_mod)

In [143]:
# get dataframe
persu_para_count = bow_mod_features(persu_para_sents, persu_para_words)
persu_para_count.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450


,company NOUN pobj,uxorious ADJ amod,interrupted ADJ acomp,uncompounded ADJ conj,clever ADJ amod,remnant NOUN dobj,home NOUN pobj,weak ADJ acomp,enormous ADJ amod,worth NOUN pobj,...,quick ADJ acomp,soft ADJ csubj,worth ADJ ccomp,pit NOUN pcomp,alarm NOUN pobj,Shapes PROPN dobj,expect VERB advcl,text_sentence,sentence_len,text_source
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,"(Sir, Walter, Elliot, ,, of, Kellynch, Hall, ,...",2,Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(This, was, the, page, at, which, the, favouri...",0,Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"("")",0,Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Walter, Elliot, ,, born, March, 1, ,, 1760, ,...",2,Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Anne, ,, born, August, 9, ,, 1787, ;, a, stil...",1,Austen


In [152]:
persu_para_count['text_source'].value_counts()

Austen    315
Milton    157
Name: text_source, dtype: int64

In [144]:
# Wow. Many columns.
# shape df accordingly
X_persu_para = persu_para_count.drop(['text_sentence', 'text_source'], axis=1).sample(2300, axis=1)
y_persu_para = persu_para_count['text_source']

print(X_persu_para.shape)
print(y_persu_para.shape)

(472, 2300)
(472,)


In [145]:
# Let's see how this dataset does with the initial model
print('Persuasion v Paradise: %0.4f' % model.score(X_persu_para, y_persu_para))

Persuasion v Paradise: 0.2034


In [149]:
y_pred2 = model.predict(X_persu_para)
pd.crosstab(y_persu_para, y_pred2)

col_0,Carroll,Milton
text_source,,
Austen,147,168
Milton,61,96
